In [ ]:
%%writefile c-means100_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

// Размерность массива
const int size_mass = 100;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 200; // Максимальное количество итераций
const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;

// Путь к файлу с данными
string file = "my_mass100.txt";

// Функция для считывания данных из файла
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
bool converged(float* new_centers, float* old_centers) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

        // Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
        if (converged(centers, oldCenters)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    cout << "Result centers:" << endl;
    // Вывод центров
    for (int i = 0; i < k; ++i) {
        for (int j = 0; j < dim; ++j) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;
}





int main() {
    float* data = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* membership = new float[size_mass * k];
    float* distances = new float[size_mass * k];

    // Считывание данных из файла
    readFromFile(data, file);

    // Указатели на GPU память
    float* dev_data;
    float* dev_centers;
    float* dev_distances;
    float* dev_membership;
    float* dev_numerator;
    float* dev_denominator;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_denominator, k * sizeof(float));

    // Копирование данных с CPU на GPU
    cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма C-Means
    c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);


    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_membership);
    cudaFree(dev_numerator);
    cudaFree(dev_denominator);

    // Освобождение памяти на CPU
    delete[] data;
    delete[] centers;
    delete[] oldCenters;
    delete[] membership;
    delete[] distances;

    return 0;
}


Writing c-means100_GPU.cu


In [ ]:
!nvcc c-means100_GPU.cu -o c-means100_GPU
# компилляция

In [ ]:
# Запуск
!./c-means100_GPU

The algorithm converged on iteration 12.

Result centers:
3.14597 2.92571 3.23767 
-3.06514 -3.23951 -2.78432 
-0.129383 -0.0903786 -0.394087 



In [ ]:
%%writefile c-means10000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

// Размерность массива
const int size_mass = 10000;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 200; // Максимальное количество итераций
const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;

// Путь к файлу с данными
string file = "my_mass10000.txt";

// Функция для считывания данных из файла
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
bool converged(float* new_centers, float* old_centers) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

        // Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
        if (converged(centers, oldCenters)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    cout << "Result centers:" << endl;
    // Вывод центров
    for (int i = 0; i < k; ++i) {
        for (int j = 0; j < dim; ++j) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;
}





int main() {
    float* data = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* membership = new float[size_mass * k];
    float* distances = new float[size_mass * k];

    // Считывание данных из файла
    readFromFile(data, file);

    // Указатели на GPU память
    float* dev_data;
    float* dev_centers;
    float* dev_distances;
    float* dev_membership;
    float* dev_numerator;
    float* dev_denominator;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_denominator, k * sizeof(float));

    // Копирование данных с CPU на GPU
    cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма C-Means
    c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);


    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_membership);
    cudaFree(dev_numerator);
    cudaFree(dev_denominator);

    // Освобождение памяти на CPU
    delete[] data;
    delete[] centers;
    delete[] oldCenters;
    delete[] membership;
    delete[] distances;

    return 0;
}


Writing c-means10000_GPU.cu


In [ ]:
!nvcc c-means10000_GPU.cu -o c-means10000_GPU
# компилляция

In [ ]:
# Запуск
!./c-means10000_GPU

The algorithm converged on iteration 9.

Result centers:
3.01316 2.97548 3.00014 
-3.06287 -3.02536 -2.99694 
-0.0218657 0.0097493 -0.0300436 



In [ ]:
%%writefile c-means100000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

// Размерность массива
const int size_mass = 100000;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 200; // Максимальное количество итераций
const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;

// Путь к файлу с данными
string file = "my_mass100000.txt";

// Функция для считывания данных из файла
void readFromFile(float* data, string fileName) {
    ifstream file(fileName);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size_mass) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    } else {
        cerr << "Error opening file: " << fileName << endl;
    }
}

// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
bool converged(float* new_centers, float* old_centers) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

        // Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
        if (converged(centers, oldCenters)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    cout << "Result centers:" << endl;
    // Вывод центров
    for (int i = 0; i < k; ++i) {
        for (int j = 0; j < dim; ++j) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;
}





int main() {
    float* data = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* membership = new float[size_mass * k];
    float* distances = new float[size_mass * k];

    // Считывание данных из файла
    readFromFile(data, file);

    // Указатели на GPU память
    float* dev_data;
    float* dev_centers;
    float* dev_distances;
    float* dev_membership;
    float* dev_numerator;
    float* dev_denominator;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_denominator, k * sizeof(float));

    // Копирование данных с CPU на GPU
    cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма C-Means
    c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);


    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_membership);
    cudaFree(dev_numerator);
    cudaFree(dev_denominator);

    // Освобождение памяти на CPU
    delete[] data;
    delete[] centers;
    delete[] oldCenters;
    delete[] membership;
    delete[] distances;

    return 0;
}


Writing c-means100000_GPU.cu


In [ ]:
!nvcc c-means100000_GPU.cu -o c-means100000_GPU
# компилляция

In [ ]:
# Запуск
!./c-means100000_GPU

The algorithm converged on iteration 9.

Result centers:
3.01037 2.99929 3.00826 
-3.02111 -3.02589 -3.02558 
-0.0141037 -0.0240736 -0.0176969 



In [ ]:
%%writefile c-means1000000_GPU.cu

#include <iostream>
#include <random>
#include <fstream>
#include <sstream>
#include <cmath>
#include <limits>
#include <chrono>
#include <cuda_runtime.h>

using namespace std;

// Размерность массива
const int size_mass = 1000000;
const int dim = 3; // размерность пространства
const int k = 3; // количество кластеров

const int m = 2; // Экспоненциальный вес
const int max_iterations = 200; // Максимальное количество итераций
const float epsilon = 0.0001; // Пороговое значение для остановки

// Объявление константной памяти на GPU
__constant__ int d_k;
__constant__ int d_dim;
__constant__ int d_m;
__constant__ int d_size_mass;



// Функция для считывания из файлов данных с размерностью (1000000, 3)
void readFromFile(float* data, int size, int part_number) {
    string filename = "my_mass1000000_part_" + to_string(part_number) + ".txt";
    ifstream file(filename);
    if (file.is_open()) {
        string line;
        int i = 0;
        while (getline(file, line) && i < size) {
            istringstream iss(line);
            float x, y, z;
            if (iss >> x >> y >> z) {
                data[i * dim + 0] = x;
                data[i * dim + 1] = y;
                data[i * dim + 2] = z;
                i++;
            }
        }
        file.close();
    }
    else {
        cerr << "Error opening file: " << filename << endl;
    }
}


// на CPU
void startingCenters(float* centers, float* data) {
    float minValues[dim];
    float maxValues[dim];

    for (int i = 0; i < dim; i++) {
        minValues[i] = numeric_limits<float>::max();
        maxValues[i] = numeric_limits<float>::lowest();
    }

    for (int i = 0; i < size_mass; i++) {
        for (int j = 0; j < dim; j++) {
            minValues[j] = min(minValues[j], data[i * dim + j]);
            maxValues[j] = max(maxValues[j], data[i * dim + j]);
        }
    }

    random_device rd;
    mt19937 gen(rd());

    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            uniform_real_distribution<> dis(minValues[j], maxValues[j]);
            centers[i * dim + j] = dis(gen);
        }
    }
}

// на GPU

// Пересчёт матрицы степеней принадлежности на основе вычисленных центроид
__global__ void updateMembershipCUDA(float* dataPoints, float* centers, float* distances, float* membership) {

    // Вычисление евклидовых расстояний от каждой точки до каждого центра

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float sum = 0.0f;
            for (int coord = 0; coord < d_dim; coord++) {
                sum += powf((dataPoints[point * d_dim + coord] - centers[num_center * d_dim + coord]), 2);
            }
            float dist = sqrtf(sum);
            distances[point * d_k + num_center] = dist;
        }

        // Вычисление степеней принадлежности для каждой точки
        for (int j = 0; j < d_k; j++) {
            float sumDistances = 0.0f;
            for (int c = 0; c < d_k; c++) {
                float distancesRatio = distances[point * d_k + j] / distances[point * d_k + c];
                sumDistances += powf(distancesRatio, (2.0f / (d_m - 1)));
            }
            membership[point * d_k + j] = 1.0f / sumDistances;
        }
    }
}

// Функция для обнуления переменных
__global__ void updateCentersCUDA_one(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                numerator[num_center * d_dim + j] = 0.0f;
            }
            denominator[num_center] = 0.0f;
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_two(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;
    if (point < d_size_mass) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            float membershipPow = powf(membership[point * d_k + num_center], d_m);
            for (int j = 0; j < d_dim; j++) {
                atomicAdd(&numerator[num_center * d_dim + j], membershipPow * data[point * d_dim + j]);
            }
            atomicAdd(&denominator[num_center], membershipPow);
        }
    }
}

// Функция для пересчёта центров
__global__ void updateCentersCUDA_three(float* data, float* centers, float* membership, float* numerator, float* denominator) {

    // Получение глобального индекса
    int point = blockIdx.x * blockDim.x + threadIdx.x;

    if (point == 0) {
        for (int num_center = 0; num_center < d_k; num_center++) {
            for (int j = 0; j < d_dim; j++) {
                centers[num_center * d_dim + j] = numerator[num_center * d_dim + j] / denominator[num_center];
            }
        }
    }
}

// Функция для проверки сходимости алгоритма
bool converged(float* new_centers, float* old_centers) {
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            if (fabs(new_centers[i * dim + j] - old_centers[i * dim + j]) >= epsilon) {
                return false;
            }
        }
    }
    return true;
}

// Основная функция алгоритма C-Means
void c_means(float* data, float* centers, float* oldCenters, float* membership, float* distances, float *dev_data, float *dev_centers, float *dev_distances, float *dev_membership, float* dev_numerator, float* dev_denominator) {

    // Копирование констант с CPU на GPU
    cudaMemcpyToSymbol(d_k, &k, sizeof(int));
    cudaMemcpyToSymbol(d_dim, &dim, sizeof(int));
    cudaMemcpyToSymbol(d_m, &m, sizeof(int));
    cudaMemcpyToSymbol(d_size_mass, &size_mass, sizeof(int));

    // Инициализация начальных центроид на CPU
    startingCenters(centers, data);

    // Перенос начальных центров на GPU
    cudaMemcpy(dev_centers, centers, k * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Копирование текущих центров в массив oldCenters
    for (int i = 0; i < k; i++) {
        for (int j = 0; j < dim; j++) {
            oldCenters[i * dim + j] = centers[i * dim + j];
        }
    }

    // Итерационный процесс
    int iteration = 0;
    while (iteration < max_iterations) {

        // Определение параметров для запуска ядер
        int blockSize = 256;
        int gridSize = (size_mass * dim + blockSize - 1) / blockSize;

        // Запуск ядра для расчёта матрицы степеней принадлежности на основе текущих центров
        updateMembershipCUDA<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_distances, dev_membership);
        cudaDeviceSynchronize();

        // Запуск ядер для пересчёта центров на основе текущей матрицы степеней принадлежности
        updateCentersCUDA_one<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_two<<<gridSize, blockSize>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        updateCentersCUDA_three<<<1, 1>>>(dev_data, dev_centers, dev_membership, dev_numerator, dev_denominator);
        cudaDeviceSynchronize();

        // Копирование новых центров с GPU на CPU
        cudaMemcpy(centers, dev_centers, k * dim * sizeof(float), cudaMemcpyDeviceToHost);

        // Условие остановки алгоритма
        if (converged(centers, oldCenters)) {
            cout << "The algorithm converged on iteration " << iteration << "." << endl << endl;
            break;
        }

        // Обновление центров
        for (int i = 0; i < k; i++) {
            for (int j = 0; j < dim; j++) {
                oldCenters[i * dim + j] = centers[i * dim + j];
            }
        }

        iteration++;
    }

    cout << "Result centers:" << endl;
    // Вывод центров
    for (int i = 0; i < k; ++i) {
        for (int j = 0; j < dim; ++j) {
            cout << centers[i * dim + j] << " ";
        }
        cout << endl;
    }
    cout << endl;
}





int main() {
    float* data = new float[size_mass * dim];
    float* centers = new float[k * dim];
    float* oldCenters = new float[k * dim];
    float* membership = new float[size_mass * k];
    float* distances = new float[size_mass * k];


    // Считывание данных из файла и запись в массив
    for (int i = 0; i <= 9; i++) {
        readFromFile(data + i * 100000, size_mass / 10, i);
    }

    // Указатели на GPU память
    float* dev_data;
    float* dev_centers;
    float* dev_distances;
    float* dev_membership;
    float* dev_numerator;
    float* dev_denominator;

    // Выделение памяти на GPU
    cudaMalloc((void**)&dev_data, size_mass * dim * sizeof(float));
    cudaMalloc((void**)&dev_centers, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_distances, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_membership, size_mass * k * sizeof(float));
    cudaMalloc((void**)&dev_numerator, k * dim * sizeof(float));
    cudaMalloc((void**)&dev_denominator, k * sizeof(float));

    // Копирование данных с CPU на GPU
    cudaMemcpy(dev_data, data, size_mass * dim * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск алгоритма C-Means
    c_means(data, centers, oldCenters, membership, distances, dev_data, dev_centers, dev_distances, dev_membership, dev_numerator, dev_denominator);


    // Освобождение памяти на GPU
    cudaFree(dev_data);
    cudaFree(dev_centers);
    cudaFree(dev_distances);
    cudaFree(dev_membership);
    cudaFree(dev_numerator);
    cudaFree(dev_denominator);

    // Освобождение памяти на CPU
    delete[] data;
    delete[] centers;
    delete[] oldCenters;
    delete[] membership;
    delete[] distances;

    return 0;
}


Overwriting c-means1000000_GPU.cu


In [ ]:
!nvcc c-means1000000_GPU.cu -o c-means1000000_GPU
# компилляция

In [ ]:
# Запуск
!./c-means1000000_GPU

The algorithm converged on iteration 7.

Result centers:
-3.03775 -3.03586 -3.03159 
3.02 3.01545 3.01568 
-0.0051878 -0.00533592 -0.0056458 

